# LSTM 비트코인 자동매매 with bithumb

### Import Libraries

In [29]:
import pyupbit
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import datetime
import schedule
import time

### Class for Bithumb

In [30]:
class Upbit_Handler:
    def __init__(self, access_key, secret_key, repl) -> None:
        # 액세스 키 설정
        self.__access_key = access_key
        self.__secret_key = secret_key
        self.__upbit = pyupbit.Upbit(self.__access_key, self.__secret_key)
        self.__REPL = repl # 종목 이름으로 변수명 변경
        
    def get_historical_price(self, ticker, interval, count):
        try:
            df = pyupbit.get_ohlcv(ticker, interval=interval, count=count)
            if df is None:
                print("No data was returned from the API.")
                return None
            else:
                return df['close']
            
        except Exception as e:
            print(f"An error occurred: {e}")
            return None

    def get_target_price(self, ticker, k):
        """변동성 돌파 전략으로 매수 목표가 조회"""
        df = pyupbit.get_ohlcv(ticker, interval="minute", count=1)
        target_price = df.iloc[0]['close'] + (df.iloc[0]['high'] - df.iloc[0]['low']) * k
        return target_price

    def get_start_time(self):
        """시작 시간 조회"""
        df = pyupbit.get_ohlcv(self.__REPL, interval="day", count=1)
        start_time = df.index[0]
        return start_time

    def get_current_price(self, ticker):
        """현재가 조회"""
        return pyupbit.get_orderbook(ticker=ticker)["orderbook_units"][0]["ask_price"]
    
    def get_balance(self, ticker):
        """잔고 조회"""
        balances = self.__upbit.get_balances()
        for b in balances:
            if b['currency'] == ticker:
                if b['balance'] is not None:
                    return float(b['balance'])
                else:
                    return 0
        return 0
    
    def buy(self, krw):
        self.__upbit.buy_market_order(self.__REPL, krw*0.9995)
        
    def buy(self, krw):
        self.__upbit.buy_market_order(self.__REPL, krw*0.9995)

In [31]:

class LSTM(Upbit_Handler): 
    def __init__(self) -> None:
        self.__scaler = MinMaxScaler()
    
    def preprocess_data(self, df):
        return self.__scaler.fit_transform(df.values.reshape(-1, 1))

    # 시퀀스 생성
    def create_sequences(self, data, seq_length):
        x, y = [], []
        for i in range(len(data) - seq_length):
            x.append(data[i:i+seq_length])
            y.append(data[i+seq_length])
        return np.array(x), np.array(y)
    
    def get_predicted_close_price(self):
        return self.__predicted_close_price
    
    def predict_price(self, ticker):
        # 비트코인 가격 데이터 받아오기
        btc_data = Upbit_Handler.get_historical_price(ticker, "minute1", 500)
        btc_prices = self.preprocess_data(btc_data)

        # 학습 데이터와 테스트 데이터 분리
        train_size = int(len(btc_prices) * 0.9)
        train_data = btc_prices[:train_size]
        test_data = btc_prices[train_size:]

        # 시퀀스 생성
        seq_length = 10  # 입력 시퀀스 길이 설정
        X_train, y_train = self.create_sequences(train_data, seq_length)
        X_test, y_test = self.create_sequences(test_data, seq_length)

        # 모델 구성
        model = tf.keras.models.Sequential([
            tf.keras.layers.LSTM(128, return_sequences=True, input_shape=(seq_length, 1)),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.LSTM(64, return_sequences=True),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.LSTM(32),
            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.Dense(1)
        ])
        
        # 모델 컴파일
        model.compile(optimizer='adam', loss='mse')
        # 모델 학습
        model.fit(X_train, y_train, epochs=5, batch_size=32, shuffle=False)
        # 예측 수행
        y_pred = model.predict(X_test)
        # 스케일 역변환
        y_pred = self.__scaler.inverse_transform(y_pred)
        y_test = self.__scaler.inverse_transform(y_test)

        plt.plot(y_test)
        plt.plot(y_pred)

        plt.savefig(str(datetime.datetime.now())+'.png')

        # 예측 결과 출력
        for i in range(len(y_pred)):
            print("실제 가격: {:.2f}, 예측 가격: {:.2f}".format(y_test[i][0], y_pred[i][0]))

        self.__predicted_close_price = y_pred[-1][0]

In [28]:

if __name__ == "__main__":
    upbit_handler = Upbit_Handler("", "", "KRW-DOGE")
    lstm = LSTM()
    
    maesu_price=0
    maesu_predicted_close=0
    flag = False

    # lstm.predict_price(REPL)
    schedule.every(10).minutes.do(lambda: lstm.predict_price())

    while True:
        start_time = upbit_handler.get_start_time()
        end_time = start_time + datetime.timedelta(days=1)
        schedule.run_pending()
        target_price = upbit_handler.get_target_price(0.6)
        current_price = upbit_handler.get_current_price()

        print('target price :', target_price, end=' | ') # k값 중요
        print('current price :', current_price, end=' | ')
        print('predicted_close_price : %.1f'%(lstm.get_predicted_close_price()))
        
        # 구매한게 없을 때
        if flag:
            if (current_price < maesu_price * 0.98) or (current_price > maesu_predicted_close):
                upbit_handler.sell(balance * 0.9995)
            maemae = 0
        # 구매한게 있을 때
        else : 
            if target_price < current_price < lstm.get_predicted_close_price():
                maesu_price = current_price
                maesu_predicted_close = lstm.get_predicted_close_price()
                balance = upbit_handler.get_balance("KRW")
                if balance > 5000:
                    upbit_handler.buy(balance * 0.9995)
            flag = 1

        time.sleep(60)

AttributeError: 'NoneType' object has no attribute 'index'